In [ ]:
# That's an impressive list of imports.
import numpy as np
import pandas as pd
import random

# We'll use matplotlib for graphics.
import matplotlib.pyplot as plt
#%matplotlib inline

import scipy.io as sio

import sklearn
from sklearn import feature_selection, datasets, model_selection, preprocessing, decomposition, metrics
from sklearn.model_selection import validation_curve, learning_curve, cross_validate, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix

import imblearn

import utils_models as um
import load_data as ld

import sys
import utils
import functions

from sklearn.metrics import roc_curve, auc
import pylab as pl
from sklearn.model_selection import train_test_split, KFold



def LR(params, folders, params_function, kfold=5):  
    matrix_all_values = np.zeros((16, len(folders)))

    for i in range(len(folders)):
        
        X_pre_train, y_pre_train, X_test, y_test = um.loadData(params_function['method1'], params_function['method2'], folders, i, params_function['fussionDataON'], params_function['data'])

        kf = KFold(n_splits=kfold, shuffle=False)
        kf.get_n_splits(X_pre_train)

        bestHyperparameters = {'C': 0}

        bestMetricDev = 0
        for c in range(len(params['C'])):
            # Construyo el modelo con los hiperparametros correspondientes
            clf = LogisticRegression(solver='liblinear', C=params['C'][c], penalty='l1', n_jobs=24)

            roc_auc_score = []
            threshold_1 = []
            threshold_2 = 0
            y_pred_arr = []
            # Divido el conjunto de test en k particiones
            for train_index, val_index in kf.split(X_pre_train):
                #print("Separo train en train' y validacion...")
                X_train, X_val = X_pre_train.iloc[train_index].reset_index(drop=True), X_pre_train.iloc[val_index].reset_index(drop=True)

                y_train, y_val = y_pre_train.iloc[train_index].reset_index(drop=True), y_pre_train.iloc[val_index].reset_index(drop=True)

                clf = um.selectCostFunction(params_function['changecostfunction'], X_train, y_train, clf)
                y_pred = clf.predict_proba(X_val)[:,1]

                threshold_1.append(um.Find_Optimal_Cutoff(y_val, y_pred))
                auc = sklearn.metrics.roc_auc_score(y_val, y_pred)
                roc_auc_score.append(auc)

                # Me guardo los resultados de y_pred
                y_pred_arr.append(y_pred)

            # Obtengo el threshold para: modelo configurado con hiperparametros con datos de train
            y_pred_ttl = um.getypredttl(y_pred_arr)
            threshold_2 = um.Find_Optimal_Cutoff(y_pre_train, y_pred_ttl)

            if np.mean(roc_auc_score) > bestMetricDev:
                print("\tCambio the best roc auc score ", bestMetricDev, " por: ", np.mean(roc_auc_score))
                bestMetricDev = np.mean(roc_auc_score)
                bestHyperparameters['C'] = params['C'][c]
                bestHyperparameters['threshold_1'] = np.mean(threshold_1)
                bestHyperparameters['threshold_2'] = threshold_2
                bestHyperparameters['y_pred_val'] = y_pred_ttl


        if params_function['plotROC_AUC_train']:
            um.plotROC_AUC_train(y_pre_train, bestHyperparameters)


        if params_function['debug']:
            print("Best roc auc score: ", bestMetricDev)
            print("C: ", bestHyperparameters["C"])
            print("threshold obtenido datos train: ", bestHyperparameters["threshold_2"])
            if (bestHyperparameters["C"] == params['C'][0]) or (bestHyperparameters["C"] == params['C'][-1]):
                print("====================================================================================")
                print("¡Atención!")
                print("Valor de C en extremo inferior o superior")
                print("====================================================================================")
            
            
        clf = LogisticRegression(solver='liblinear', C=bestHyperparameters['C'],  penalty='l1', n_jobs=24)
        clf = um.selectCostFunction(params_function['changecostfunction'], X_pre_train, y_pre_train, clf)
        
        threshold_3, y_pred_test = um.plotROC_AUC_test(X_test, y_test, params_function['plotROC_AUC_test'], clf)
        
        if params_function['plotConfussionMatrix']:
            # PINTO LA MATRIZ DE CONFUSION
            print("Threshold data train")
            y_pred = (y_pred_test > bestHyperparameters["threshold_2"]).astype('int')
            um.plotConfusionMatrix(y_test, y_pred)
            print("Threshold 0.5")
            y_pred = (y_pred_test > 0.5).astype('int')
            um.plotConfusionMatrix(y_test, y_pred)

        y_pred_Xtrain = clf.predict(X_pre_train)
        
        selecCalculateMetrics_aux = ['umbral: 0.5' , 
                                     'umbral: threshold_2 (datos train)']
                        
        for j in range(len(selecCalculateMetrics_aux)):
            
            v_accuracy_test = []
            v_accuracy_train = []
            v_specificity = []
            v_sensitivity = []
            v_precision = []
            v_recall = []
            v_f1score = []
            v_accuracy = []
            auc_score = []

            if selecCalculateMetrics_aux[j] == 'umbral: 0.5':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > 0.5).astype('int')
            elif selecCalculateMetrics_aux[j] == 'umbral: threshold_2 (datos train)':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > bestHyperparameters["threshold_2"]).astype('int')
            elif selecCalculateMetrics_aux[j] == 'umbral: threshold_3 (datos test)':
                auc_score.append(sklearn.metrics.roc_auc_score(y_test, y_pred_test))
                y_pred = (y_pred_test > threshold_3).astype('int')

            v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train = utils.calculateconfusionmatrix(y_pred, y_pre_train, y_test, v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train, 0, y_pred_Xtrain)
            #print(matrix_all_values)
            matrix_all_values[j*8:j*8 + 8, i] = v_specificity[0], v_recall[0], v_f1score[0], v_precision[0], v_accuracy[0], v_accuracy_test[0], v_accuracy_train[0], auc_score[0]
                                 
            if params_function['printResultsbyThreshold']: 
                utils.printOutAlgorithm(v_specificity, v_recall, v_f1score, v_precision, v_accuracy, v_accuracy_test, v_accuracy_train, auc_score)
        
   
    print("====> CON UMBRAL OBTENIDO CON LOS DATOS DE TRAIN")
    print()
    utils.printOutAlgorithm(matrix_all_values[8,:], matrix_all_values[9,:], matrix_all_values[10,:], matrix_all_values[11,:], matrix_all_values[12,:], matrix_all_values[13,:], matrix_all_values[14,:], matrix_all_values[15,:])

    if params_function['printThresholdTrainAnd05']:
        print()
        print("====> CON UMBRAL: 0.5")
        print()
        utils.printOutAlgorithm(matrix_all_values[0,:], matrix_all_values[1,:], matrix_all_values[2,:], matrix_all_values[3,:], matrix_all_values[4,:], matrix_all_values[5,:], matrix_all_values[6,:], matrix_all_values[7,:])
    
       
    return matrix_all_values